In [ ]:
import numpy as np
import pandas as pd
from numba import njit
import vectorbtpro as vbt
vbt.settings.set_theme("dark")
vbt.settings.plotting["layout"]["width"] = 800
vbt.settings.plotting['layout']['height'] = 200
import warnings
warnings.filterwarnings("ignore")

import pandas_ta as ta

In [ ]:
btc_90M_db_vbt = vbt.BinanceData.load('data/btc_90M_db_vbt.pkl')

data = btc_90M_db_vbt['2021-01-01':'2023-01-01']
outofsample_data = btc_90M_db_vbt['2023-01-01':'2023-06-03']
print(data.shape)
print(outofsample_data.shape)
# Wherever you saved the pickle file
data_path = '/Users/ericervin/Documents/Coding/data-repository/data/fixed_BTCUSDT.csv'
min_data = vbt.BinanceData.from_csv(data_path)
print(min_data.shape)

# Build a PSAR function
This function can also be used to upsample and downsample the data

In [ ]:
min_data.get()

In [ ]:
def get_psar_signal(high, low, close, af0=0.02, step=0.02, max_=0.2, resample_period=None):
    """
    Returns a DataFrame with the following columns:
    - 'signal': 1 (long) or -1 (short)
    - 'close_long_price': price at which a long position should be closed
    - 'close_short_price': price at which a short position should be closed
    
    """
    data = pd.concat([high, low, close], axis=1)
    data.columns = ['High', 'Low', 'Close']

    # Resample data if resample_period is provided
    if resample_period:
        data_resampled = data.resample(resample_period).agg({'High': 'max', 'Low': 'min', 'Close': 'last'})
    else:
        data_resampled = data
    
    psar = data_resampled.ta.psar(af0, step, max_)

    close_long_price = f"PSARl_{af0}_{max_}"   # or 'floor'
    close_short_price = f"PSARs_{af0}_{max_}"  # or 'ceiling'
    psar_reversal_col = f"PSARr_{af0}_{max_}"

    signal = np.zeros(len(psar))
    signal = np.where((psar[psar_reversal_col] == 1) & (psar[close_long_price].shift(1).notna()), 1, signal)  # breakout to the upside
    signal = np.where((psar[psar_reversal_col] == 1) & (psar[close_short_price].shift(1).notna()), -1, signal)  # breakout to the downside

    result = pd.DataFrame({
        'Close': data_resampled.Close,
        'signal': signal,
        'close_long_price': psar[close_long_price],
        'close_short_price': psar[close_short_price]
    }, index=data_resampled.index)

    # Reindex to the original timeframe and forward fill if resampling was done
    if resample_period:
        result = result.reindex(data.index).ffill()
        
        # Replace NaN values in the signal column with 0 (after resampling)
        result['signal'].fillna(0, inplace=True)

    return result


psar_signal = get_psar_signal(
    min_data.high, 
    min_data.low, 
    min_data.close, 
    resample_period='1h')
# print a small sample of the result
plot_columns = ['close_long_price', 'close_short_price']
fig = min_data.loc['2019-1-01':'2019-01-31'].close.vbt.plot()
psar_signal[plot_columns].loc['2019-01-01':'2019-01-31'].vbt.plot(fig=fig).show()


In [ ]:
psar_signal = get_psar_signal(min_data.high, min_data.low, min_data.close, resample_period='1h')

In [ ]:
psar_signal[10000:10050]


In [ ]:
test_minutes = min_data.get()
test_minutes.ta.psar(0.02, 0.02, 0.2, append=True)

In [ ]:
test_minutes[100:150]

In [ ]:
test_minutes[100:10000][['Open','High','Low','Close', 'PSARl_0.02_0.2', 'PSARs_0.02_0.2',]].vbt.plot().show()

In [ ]:
psar_signal[50:100]

if close_long_price is not nan then a bullish trend is in place

In [ ]:
psar_pf = vbt.Portfolio.from_signals(
    close           =data.close,
    high            =data.high,
    low             =data.low,
    open            =data.open, 
    entries         =psar_signal['signal'] == 1, 
    exits           =psar_signal['signal'] == -1,
    short_entries   =psar_signal['signal'] == -1,
    short_exits     =psar_signal['signal'] == 1,
    # tsl_th          =0.0050,
    # tsl_stop        =0.0015,
    )
print(psar_pf.stats())

# Mean Reversion
Reverse the signals, if psar crosses up short it and if it crosses down get long


In [ ]:
psar_pf = vbt.Portfolio.from_signals(
    close           =data.close,
    high            =data.high,
    low             =data.low,
    open            =data.open, 
    entries         =psar_signal['signal'] == -1, 
    exits           =psar_signal['signal'] == 1,
    short_entries   =psar_signal['signal'] == 1,
    short_exits     =psar_signal['signal'] == -1,
    tsl_th          =0.003,
    tsl_stop        =0.0015,
    )
print(psar_pf.stats())
# psar_pf.plot().show()

In [ ]:
psar_pf.plot().show()

## Now let's compare these same versions on minutely data

In [ ]:
start = '2019-01-01'
end = '2023-09-30'
psar_signal = get_psar_signal(
    min_data.loc[start:end].high, 
    min_data.loc[start:end].low, 
    min_data.loc[start:end].close, 
    resample_period='2h')

In [ ]:
# psar_signal.vbt.plot().show()

In [ ]:
psar_pf = vbt.Portfolio.from_signals(
    close           =min_data.loc[start:end].close,
    high            =min_data.loc[start:end].high,
    low             =min_data.loc[start:end].low,
    open            =min_data.loc[start:end].open, 
    entries         =psar_signal['signal'] == 1, 
    exits           =psar_signal['signal'] == -1,
    short_entries   =psar_signal['signal'] == -1,
    short_exits     =psar_signal['signal'] == 1,
    # tsl_th          =0.003,
    # tsl_stop        =0.0015,
    freq            ='1m',
    fees            =0.0005,
    # sl_stop         =0.01,
    leverage        =1,
    )
print(psar_pf.stats())
# psar_pf.resample('1d').plot().show()

In [ ]:
psar_signal['signal'].value_counts()

In [ ]:
psar_signal.loc[start:'2019-01-31'][['Close','close_long_price', 'close_short_price']].vbt.plot().show()

# Reverse the signals
mean reversion version

In [ ]:
psar_pf = vbt.Portfolio.from_signals(
    close           =min_data.loc[start:end].close,
    high            =min_data.loc[start:end].high,
    low             =min_data.loc[start:end].low,
    open            =min_data.loc[start:end].open, 
    entries         =psar_signal['signal'] == -1, 
    exits           =psar_signal['signal'] == 1,
    short_entries   =psar_signal['signal'] == 1,
    short_exits     =psar_signal['signal'] == -1,
    # tsl_th          =0.003,
    # tsl_stop        =0.0015,
    freq            ='1m',
    fees            =0.0014,
    # sl_stop         =0.01,
    leverage        =1,
    )
print(psar_pf.stats())
# psar_pf.resample('1d').plot().show()

In [ ]:
psar_pf.resample('6h').plot().show()

In [ ]:
min_data['2019-10-26':'2019-10-27 00:40:00'][['Open', 'High', 'Low', 'Close']].plot().show()

In [ ]:
min_data.loc['2020-03-12'].close.vbt.plot().show()

In [ ]:
# order by return
psar_pf.trades.records_readable.sort_values('Return', ascending=False).head(10)

In [ ]:
psar_pf.trades.records_readable

# Now let's set it up for hyperparamater optimization

In [ ]:
def get_psar_signal(high, low, close, af0=0.02, step=0.02, max_=0.2, resample_period=None):
    """
    Compute PSAR signals with optional resampling.
    
    Args:
    ... [same docstring arguments as before] ...
    
    - resample_period (str, optional): If provided, the data will be resampled to this period. E.g. '2H' for 2 hours.

    Returns:
    - DataFrame containing:
      * signal: buy signals (1), sell signals (-1), and no action (0).
      * close_long_price: Level at which a long position should be closed or reversed to short.
      * close_short_price: Level at which a short position should be closed or reversed to long.
    """
    # The next 3 lines help to work with numpy arrays because vbt converts them to numpy arrays

    high = pd.Series(high)
    low = pd.Series(low)
    close = pd.Series(close)
    
    data = pd.concat([high, low, close], axis=1)
    data.columns = ['High', 'Low', 'Close']

    # Resample data if resample_period is provided
    if resample_period:
        data_resampled = data.resample(resample_period).agg({'High': 'max', 'Low': 'min', 'Close': 'last'})
    else:
        data_resampled = data
    
    psar = data_resampled.ta.psar(af0, step, max_)

    close_long_price = f"PSARl_{af0}_{max_}"   # or 'floor'
    close_short_price = f"PSARs_{af0}_{max_}"  # or 'ceiling'
    psar_reversal_col = f"PSARr_{af0}_{max_}"

    signal = np.zeros(len(psar))
    signal = np.where((psar[psar_reversal_col] == 1) & (psar[close_long_price].shift(1).notna()), 1, signal)  # buy signal
    signal = np.where((psar[psar_reversal_col] == 1) & (psar[close_short_price].shift(1).notna()), -1, signal)  # sell signal

    result = pd.DataFrame({
        'signal': signal,
        'close_long_price': psar[close_long_price],
        'close_short_price': psar[close_short_price]
    }, index=data_resampled.index)
    
    # Reindex to the original timeframe and forward fill if resampling was done
    if resample_period:
        result = result.reindex(data.index).ffill()
        
        # Replace NaN values in the signal column with 0 (after resampling)
        result['signal'].fillna(0, inplace=True)

    return result["signal"], result["close_long_price"], result["close_short_price"]    

psar_indiator = vbt.IndicatorFactory(
    class_name='ParabolicSAR',
    short_name='psar',
    input_names=['high', 'low', 'close'],
    param_names=['af0', 'step', 'max_'],
    output_names=['signal','close_long_price', 'close_short_price'],
).with_apply_func(
    get_psar_signal,
    takes_1d=True,
    af0     =0.02,
    step    =0.02,
    max_    =0.2,
    resample_period=None,
)
psar_combinations = psar_indiator.run(
    data.high,
    data.low,
    data.close,
    af0     =np.arange(0.018,   0.022,   0.001),
    step    =0.02, #np.arange(0.02,   0.04,   0.01),
    max_    =np.arange(0.2,   0.25,  0.01),
    param_product=True,
        execute_kwargs=dict(
        engine="threadpool",
        chunk_len="auto",
        show_progress=True,
    )
)
   


In [ ]:
pf = vbt.Portfolio.from_signals(
    close=  data.high,
    high=   data.high,
    low=    data.low,
    entries =       psar_combinations.signal==-1,
    exits =         psar_combinations.signal==1,
    short_entries = psar_combinations.signal==1,
    short_exits =   psar_combinations.signal==-1,
    freq = '10m',
    # tp_stop=0.003,
    tsl_th = 0.003,
    tsl_stop=0.0015,
    # sl_stop=0.02,

)
print(pf.stats())


In [ ]:
# print the total returns for all the combinations
print(f'The best total return is {pf.total_return.max()}')

# Isolate the best Sharpe ratio portfolio
best_sharpe = pf.sharpe_ratio.max()
print(f'The best Sharpe ratio of all the combinations is {best_sharpe:.2f}')
best_sharpe_combination = pf.sharpe_ratio.idxmax()
print(f'The best combination is {best_sharpe_combination}')

# Isolate the best Sortino ratio portfolio
best_sortino = pf.sortino_ratio.max()
print(f'The best Sortino ratio of all the combinations is {best_sortino:.2f}')
best_sortino_combination = pf.sortino_ratio.idxmax()
print(f'The best combination is {best_sortino_combination}')

# Isolate the best Win rate portfolio
best_win_rate = pf.trades.win_rate.max() # Note these are in the portfolio.trades object not the portfolio object
print(f'The best Win rate of all the combinations is {best_win_rate:.2f}')
best_win_rate_combination = pf.trades.win_rate.idxmax() 
print(f'The best combination is {best_win_rate_combination}')

# Isolate the best max drawdown
best_max_drawdown = pf.max_drawdown.max()
print(f'The best max drawdown of all the combinations is {best_max_drawdown:.2%}')
best_max_drawdown_combination = pf.max_drawdown.idxmax()
print(f'The best combination is {best_max_drawdown_combination}')

# You get the gist. You can do this for any of the metrics in the stats dataframe

# Show the portfolio backtest simulation
# pf[13,9].plot().show() # you can call the pf object like a dictionary to get the backtest of a specific combination
# The above is the same as 
pf[best_sharpe_combination].plot().show() # you can call the pf object like a dictionary to get the backtest of a specific combination

# Show the portfolio backtest simulation

In [ ]:
pf.sharpe_ratio.vbt.volume().show()